## Library Setup

transformers version > 3.0.0 is needed to load the model

In [ ]:
!pip install transformers==3.0.0
!pip install nlp==0.2.0
!pip install datasets

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import transformers
import nlp
import logging
logging.basicConfig(level=logging.INFO)
from datasets import load_dataset

## Loading the data

In [ ]:
data = load_dataset(r'/content/comsen_loader.py')
dataset_dict = {"commonsense_qa": data}  

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['answerKey', 'statement', 'choices'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['answerKey', 'statement', 'choices'],
        num_rows: 997
    })
    test: Dataset({
        features: ['answerKey', 'statement', 'choices'],
        num_rows: 1000
    })
})

## Loading the saved model

In [ ]:
# loading the saved model 
model = transformers.AutoModelForMultipleChoice.from_pretrained(
                pretrained_model_name_or_path = "/content/drive/MyDrive/model", 
                config=transformers.AutoConfig.from_pretrained('roberta-base'),
            )

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

INFO:transformers.modeling_utils:loading weights file /content/drive/MyDrive/model/pytorch_model.bin

## Testing the model

### example from the test set

In [ ]:
from transformers import RobertaTokenizer
model.eval()
model.cuda()

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
list=[]
for i in range(len(data['test'])):
  prompt_test = data['test'][i]['statement']
  choice0_test = data['test'][i]['choices']['text'][0]
  choice1_test = data['test'][i]['choices']['text'][1]
  choice2_test = data['test'][i]['choices']['text'][2]

  labels = torch.tensor(0).unsqueeze(0) 

  encoding = tokenizer([prompt_test, prompt_test, prompt_test], 
                     [choice0_test, choice1_test, choice2_test],
                     return_tensors='pt', padding=True)
  outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels,)  # batch size is 1

RuntimeError: ignored

In [ ]:
encoding['input_ids']

tensor([[    0,   894,   342,    10, 13280,    88,     5,  5730,     2,     2,
           642, 15409,    32,  2333,  6907,   150, 31592,    32,  2333,  4334,
             2],
        [    0,   894,   342,    10, 13280,    88,     5,  5730,     2,     2,
           102, 13280,    16,   203,  2671,    87,    10,  5730,     2,     1,
             1],
        [    0,   894,   342,    10, 13280,    88,     5,  5730,     2,     2,
           642, 15409,  1395,   304,    10,  5730,     2,     1,     1,     1,
             1]])

In [ ]:
data['test'][i]

{'answerKey': 'B',
 'choices': {'label': ['A', 'B', 'C'],
  'text': ['pigs are usually pink while pans are usually silver',
   'a pig is much bigger than a pan',
   'pigs cannot use a pan']},
 'statement': 'He put a pig into the pan'}

In [ ]:
outputs

(tensor(1.1069, grad_fn=<NllLossBackward>),
 tensor([[0.2693, 0.2609, 0.2526]], grad_fn=<ViewBackward>))

In [ ]:
reasons= [choice0_test,choice1_test,choice2_test]
a = [outputs[1][0][0],outputs[1][0][1],outputs[1][0][2]]
m = max(a)
id = [i for i, j in enumerate(a) if j == m]
choice_index = id[0]
print('statement : ',prompt_test)
print('the right reason :',reasons[choice_index])

statement :  The chef put extra lemons on the pizza.
the right reason : Many types of lemons are to sour to eat.


In [ ]:
reasons[choice_index]

'It is eathen with a napkin.'

### New Example

In [ ]:
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
choice0 = "It is eaten with a fork and a knife."
choice1 = "It is eaten while held in the hand."
choice2 = "It is eathen with a napkin."

labels = torch.tensor(0).unsqueeze(0) 

encoding = tokenizer([prompt, prompt, prompt], 
                     [choice0, choice1, choice2],
                     return_tensors='pt', padding=True)
outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels)  # batch size is 1

In [ ]:
outputs

(tensor(1.0991, grad_fn=<NllLossBackward>),
 tensor([[0.6448, 0.6393, 0.6519]], grad_fn=<ViewBackward>))

In [ ]:
reasons= [choice0,choice1,choice2]
a = [outputs[1][0][0],outputs[1][0][1],outputs[1][0][2]]
m = max(a)
id = [i for i, j in enumerate(a) if j == m]
choice_index = id[0]
print('statement : ',prompt)
print('the right reason :',reasons[choice_index])

statement :  In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.
the right reason : It is eathen with a napkin.
